# Test backend (pose graph)

In [ ]:
import numpy as np
from graphslam.load import load_g2o_se3
from graphslam.graph import Graph
from graphslam.vertex import Vertex
from graphslam.pose.se3 import PoseSE3
from graphslam.edge.edge_odometry import EdgeOdometry
import plotly.graph_objects as go

from lgchimera.geom_util import R_to_quat
from lgchimera.pose_graph import PoseGraph

%load_ext autoreload
%autoreload 2

## Graph formation

In [ ]:
g = PoseGraph()

GPS_SIGMA = 0.1
LIDAR_SIGMA = 0.1

GPS_cov = np.eye(6)
GPS_cov[:3,:3] *= GPS_SIGMA
LiDAR_cov = np.eye(6)
LiDAR_cov[:3,:3] *= LIDAR_SIGMA
GPS_inf = np.linalg.inv(GPS_cov)
LiDAR_inf = np.linalg.inv(LiDAR_cov)

# Initial node positions
N = 5
node_positions = np.array([[0, 0, 0],
                           [1, 0, 0],
                           [2, 0, 0],
                           [3, 0, 0],
                           [4, 0, 0]], dtype=float)

# GPS position measurements
GPS_positions = node_positions.copy()
GPS_positions += np.random.normal(0, GPS_SIGMA, (N,3))

In [ ]:
# Nodes
for i, pos in enumerate(node_positions):
    g.add_node(i+1, (np.eye(3), pos))

# Lidar odometry edges
# R = I for all edges
lidar_ts = np.diff(node_positions, axis=0) + np.random.normal(0, LIDAR_SIGMA, (N-1,3))
lidar_R = np.eye(3)
for i, t in enumerate(lidar_ts):
    g.add_edge([i+1, i+2], (lidar_R, t), information=LiDAR_inf)

# GPS factors
for i, pos in enumerate(GPS_positions):
    g.add_factor(i+1, (np.eye(3), pos), information=GPS_inf)

In [ ]:
fig = go.Figure(data=g.plot_trace())
fig.update_layout(width=1000, height=800, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
[e.vertex_ids for e in g.graph._edges]

In [ ]:
fig = go.Figure(data=g.plot_trace())
fig.update_layout(width=1000, height=800, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
e = g.graph._edges[4]
e.calc_error()

In [ ]:
g.optimize(suppress_output=False)

In [ ]:
g.get_positions()

In [ ]:
fig = go.Figure(data=g.plot_trace())
fig.update_layout(width=1600, height=900, scene=dict(aspectmode='data'))
fig.show()

## "Spoofed" GPS measurements

In [ ]:
GPS_positions = node_positions.copy()
GPS_positions += np.random.normal(0, GPS_SIGMA, (N,3))
GPS_positions[2,:] += [0, 2, 0]

In [ ]:
g = PoseGraph()

# Nodes
for i, pos in enumerate(node_positions):
    g.add_node(i+1, (np.eye(3), pos))

# Lidar odometry edges
# R = I for all edges
lidar_ts = np.diff(node_positions, axis=0) + np.random.normal(0, LIDAR_SIGMA, (N-1,3))
lidar_R = np.eye(3)
for i, t in enumerate(lidar_ts):
    g.add_edge([i+1, i+2], (lidar_R, t), information=LiDAR_inf)

# GPS factors
for i, pos in enumerate(GPS_positions):
    g.add_factor(i+1, (np.eye(3), pos), information=GPS_inf)

In [ ]:
fig = go.Figure(data=g.plot_trace())
fig.update_layout(width=1600, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
g.optimize()

In [ ]:
fig = go.Figure(data=g.plot_trace())
fig.update_layout(width=1600, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Compute test statistic
q = 0
for e in g.graph._edges:
    if e.vertex_ids[0] > 0:
        q += e.calc_chi2()

In [ ]:
q

In [ ]:
from scipy.stats import chi2

# Compute threshold
alpha = 0.001
T = chi2.ppf(1-alpha, df=3*N)

In [ ]:
T

## Position Edges

In [ ]:
e = g.graph._edges[0]
e.calc_error()

In [ ]:
from lgchimera.edge_position import EdgePosition
from graphslam.pose.r3 import PoseR3

In [ ]:
p1 = PoseSE3(np.zeros(3), np.zeros(4))
v1 = Vertex(10, p1)
p2 = PoseSE3(np.zeros(3), np.zeros(4))
v2 = Vertex(10, p2)
PoseR3(np.zeros(3)) - v1.pose[:3].to_array() - v2.pose[:3].to_array()

In [ ]:
e = EdgePosition([0, 1], information=np.eye(3), estimate=np.zeros(3))
e.calc_error()

In [ ]:
g.graph._link_edges()